In [2]:
import pandas as pd
import quantaxis_ext
from QUANTAXIS import QA_fetch_stock_day_adv
from QUANTAXIS import QA_fetch_index_day_adv
import ploter
import talib
import numpy as np
import settings
import units
import header
import calculator as calc
import matplotlib.pyplot as plt

#设定绘图的默认大小
import matplotlib
matplotlib.rcParams["figure.figsize"]=[16,5]

#加载 seaborn，并且设置默认使用 seaborn
import seaborn as sns
sns.set()

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Noto Sans CJK SC','SimHei']
matplotlib.rcParams['axes.unicode_minus']=False #用来正常显示负号

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

QUANTAXIS>> Expand macros in c:\program files\python37\lib\site-packages\macropy\core\hquotes.py
QUANTAXIS>> Finding macros in 'macropy.core.hquotes'
QUANTAXIS>> Importing macros from 'macropy.core.quotes' into 'macropy.core.hquotes'
QUANTAXIS>> Expand macros in c:\program files\python37\lib\site-packages\macropy\core\quotes.py
QUANTAXIS>> Finding macros in 'macropy.core.quotes'
QUANTAXIS>> Expand macros in c:\program files\python37\lib\site-packages\macropy\core\failure.py
QUANTAXIS>> Finding macros in 'macropy.core.failure'
QUANTAXIS>> Importing macros from 'macropy.core.hquotes' into 'macropy.core.failure'
QUANTAXIS>> Expand macros in c:\program files\python37\lib\site-packages\cffi\api.py
QUANTAXIS>> Finding macros in 'cffi.api'
QUANTAXIS>> Expand macros in c:\program files\python37\lib\site-packages\scipy\__config__.py
QUANTAXIS>> Finding macros in 'scipy.__config__'


In [3]:
START_VALS = 1000000 # 初始投资总额
START = '2018-01-01' # 初始投资日期
END = '2018-12-31' # 结束投资日期
# 投资组合：贵州茅台、中国平安、招商银行、格力电器
SYMBOLS = ['600519','601318','600036','000651'] 
# 比率分配
ALLOCS = [0.4,0.4,0.1,0.1]
ZS_CODE='000300'#指数代码。沪深300

## 组合日回报率

In [8]:
daily_return=quantaxis_ext.fetch_index_stock_daily_adv(SYMBOLS, ZS_CODE, START, END).pct_change().dropna()
daily_return.head()

,zs_000300,600519,601318,600036,000651
date,,,,,
2018-01-03,0.005869,0.017063,-0.022317,0.011816,0.011074
2018-01-04,0.004237,0.029629,0.002959,-0.010677,0.020372
2018-01-05,0.002407,0.001750,-0.004074,0.015177,0.027909
2018-01-08,0.005173,0.018649,-0.012414,-0.020930,-0.001044
2018-01-09,0.007005,0.040405,0.026568,0.010180,0.032825


## 组合平均日回报率

In [13]:
daily_return.mean()

zs_000300   -0.001172
600519      -0.000476
601318      -0.000867
600036      -0.000483
000651      -0.000706
dtype: float64

## 协方差矩阵

In [14]:
daily_return.cov()

,zs_000300,600519,601318,600036,000651
zs_000300,0.000182,0.000224,0.000224,0.000184,0.000233
600519,0.000224,0.000506,0.000297,0.000225,0.000374
601318,0.000224,0.000297,0.000396,0.000284,0.000322
600036,0.000184,0.000225,0.000284,0.000370,0.000249
000651,0.000233,0.000374,0.000322,0.000249,0.000532


In [1]:
import cvxopt as opt
import cvxopt.solvers as optsolvers

ModuleNotFoundError: No module named 'cvxopt'

## 计算最小方差组合

In [ ]:
def min_var_portfolio(cov_mat, allow_short=False):
    """
    Computes the minimum variance portfolio.
    Note: As the variance is not invariant with respect
    to leverage, it is not possible to construct non-trivial
    market neutral minimum variance portfolios. This is because
    the variance approaches zero with decreasing leverage,
    i.e. the market neutral portfolio with minimum variance
    is not invested at all.
    
    Parameters
    ----------
    cov_mat: pandas.DataFrame
        Covariance matrix of asset returns.
    allow_short: bool, optional
        If 'False' construct a long-only portfolio.
        If 'True' allow shorting, i.e. negative weights.
    Returns
    -------
    weights: pandas.Series
        Optimal asset weights.
    """
    if not isinstance(cov_mat, pd.DataFrame):
        raise ValueError("Covariance matrix is not a DataFrame")

    n = len(cov_mat)

    P = opt.matrix(cov_mat.values)
    q = opt.matrix(0.0, (n, 1))

    # Constraints Gx <= h
    if not allow_short:
        # x >= 0
        G = opt.matrix(-np.identity(n))
        h = opt.matrix(0.0, (n, 1))
    else:
        G = None
        h = None

    # Constraints Ax = b
    # sum(x) = 1
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    # Solve
    optsolvers.options['show_progress'] = False
    sol = optsolvers.qp(P, q, G, h, A, b)

    if sol['status'] != 'optimal':
        warnings.warn("Convergence problem")

    # Put weights into a labeled series
    weights = pd.Series(sol['x'], index=cov_mat.index)
    return weights